In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import gzip
import sys
import os
import zarr
import vcf
import allel
from scipy.stats import chi2_contingency

In [ ]:
# Set up data folder
DATA = Path('/content/drive/My Drive/data')

## Sub-population specific variants

The first 100,000 variants from chromosome 11 in 1000 Genomes Project.

In [0]:
csv_filename = DATA /'1kgn_phase3_population.csv'
csv = pd.read_csv(csv_filename)
csv.head()

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Data collections
0,HG00099,female,SAME123271,GBR,British,EUR,European,"1000 Genomes on GRCh38,1000 Genomes phase 3 re..."
1,HG00096,male,SAME123268,GBR,British,EUR,European,"1000 Genomes on GRCh38,1000 Genomes phase 3 re..."
2,HG00102,female,SAME123945,GBR,British,EUR,European,"1000 Genomes on GRCh38,1000 Genomes phase 3 re..."
3,HG00143,male,SAME124393,GBR,British,EUR,European,"1000 Genomes on GRCh38,1000 Genomes phase 3 re..."
4,HG00107,male,SAME123947,GBR,British,EUR,European,"1000 Genomes on GRCh38,1000 Genomes phase 3 re..."


In [0]:
sp = csv['Superpopulation code'].unique()
print(f"There are {len(sp)} superpopulations.")
sp

There are 5 superpopulations.


array(['EUR', 'EAS', 'AMR', 'SAS', 'AFR'], dtype=object)

In [0]:
csv['Superpopulation name'].unique()

array(['European', 'East Asian', 'American', 'South Asian', 'African'],
      dtype=object)

In [0]:
sp_num = []
sp_name = []
for p in sp:
  csv_sp = csv[csv['Superpopulation code'] == p]
  sp_num.append(len(csv_sp))
  sp_name.append(csv_sp['Superpopulation name'].iloc[0])
  print(sp_name[-1], ':', sp_num[-1])

European : 503
East Asian : 504
American : 347
South Asian : 489
African : 661


In [0]:
# !gzip -cd /content/drive/My\ Drive/data/ALL.chr11.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.first100k.vcf.gz | head -n 300

In [2]:
vcf_file= DATA/"ALL.chr11.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.first100k.vcf.gz"

# Show all available fields in one row of VCF
vcf_handle = vcf.Reader(open(vcf_file, "rb"))
rec = vcf_handle.__next__()

for key in dir(rec):
    if not key.startswith("__"):
        if not callable(getattr(rec, key)):
            print("{}: {}".format(key, getattr(rec, key)))          


In [0]:
vcf = allel.read_vcf(str(vcf_file), ["variants/ID", "variants/numalt"],
                     log=sys.stdout)

[read_vcf] 65536 rows in 4.40s; chunk in 4.40s (14887 rows/s); 11 :1963136
[read_vcf] 100000 rows in 6.70s; chunk in 2.29s (15020 rows/s)
[read_vcf] all done (14930 rows/s)


In [0]:
numalt = vcf['variants/numalt']
max_alt = np.max(numalt)

In [0]:
# clear the variable if it already exists to be gentle on memory
if "vcf" in dir():
    del vcf

zarr_file = vcf_file.name.replace(".vcf.gz", ".zarr")
if not os.path.exists(zarr_file):
    allel.vcf_to_zarr(str(vcf_file),
                      str(zarr_file),
                      group='11', fields='*', alt_number=max_alt,
                      log=sys.stdout, overwrite=True)

[vcf_to_zarr] 65536 rows in 15.16s; chunk in 15.16s (4321 rows/s); 11 :1963136
[vcf_to_zarr] 100000 rows in 23.80s; chunk in 8.64s (3990 rows/s)
[vcf_to_zarr] all done (4138 rows/s)


In [0]:
!ls

adc.json
ALL.chr11.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.first100k.zarr
drive
sample_data


In [0]:
vcfzarr = zarr.open_group(zarr_file, mode="r")
vcfzarr.tree(expand=False)

/
 └── 11
     ├── calldata
     │   └── GT (100000, 2504, 2) int8
     ├── samples (2504,) object
     └── variants
         ├── AA (100000,) object
         ├── AC (100000, 5) int32
         ├── AF (100000, 5) float32
         ├── AFR_AF (100000, 5) float32
         ├── ALT (100000, 5) object
         ├── AMR_AF (100000, 5) float32
         ├── AN (100000,) int32
         ├── CHROM (100000,) object
         ├── CIEND (100000, 2) int32
         ├── CIPOS (100000, 2) int32
         ├── CS (100000,) object
         ├── DP (100000,) int32
         ├── EAS_AF (100000, 5) float32
         ├── END (100000,) int32
         ├── EUR_AF (100000, 5) float32
         ├── EX_TARGET (100000,) bool
         ├── FILTER_PASS (100000,) bool
         ├── ID (100000,) object
         ├── IMPRECISE (100000,) bool
         ├── MC (100000,) object
         ├── MEINFO (100000, 4) object
         ├── MEND (100000,) int32
         ├── MLEN (100000,) int32
         ├── MSTART (100000,) int32
         ├── MULTI_ALLELIC (100000,) bool
         ├── NS (100000,) int32
         ├── POS (100000,) int32
         ├── QUAL (100000,) float32
         ├── REF (100000,) object
         ├── SAS_AF (100000, 5) float32
         ├── SVLEN (100000,) int32
         ├── SVTYPE (100000,) object
         ├── TSD (100000,) object
         ├── VT (100000,) object
         ├── altlen (100000, 5) int32
         ├── is_snp (100000,) bool
         └── numalt (100000,) int32

In [0]:
group = vcfzarr['11/variants/EAS_AF'][:]
# group

In [0]:
group = pd.DataFrame(group)
group0 = group.replace(np.NaN, 0)
# group0.head()

In [0]:
group_max = group0.max(axis = 1)
group_max.head()

0    0.9921
1    0.0000
2    0.0000
3    0.0030
4    0.0000
dtype: float32

In [0]:
threshold = 0.5
loc_group_select = group_max >= threshold
loc_group_select.head()

0     True
1    False
2    False
3    False
4    False
dtype: bool

In [0]:
gt = allel.GenotypeArray(vcfzarr["11/calldata/GT"])
gt

<GenotypeArray shape=(100000, 2504, 2) dtype=int8>
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
1/1 1/1 0/1 1/0 1/1 ... 1/1 1/1 1/1 1/1 1/1

In [0]:
gt_select = gt[loc_group_select]
gt_select

<GenotypeArray shape=(2861, 2504, 2) dtype=int8>
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 1/1 0/0
1/1 0/0 0/0 0/0 0/1 ... 1/1 1/1 1/0 1/1 0/0
...
1/1 0/0 0/1 0/0 1/1 ... 0/1 1/1 0/1 1/1 1/1
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
1/1 1/1 0/1 1/0 1/1 ... 1/1 1/1 1/1 1/1 1/1

In [0]:
alt_count = gt_select.count_alleles()

variant_id = np.array(vcfzarr["11/variants/ID"])

ac = pd.DataFrame(gt_select.count_alleles())
ac.index = np.array(variant_id)[loc_group_select]

print("number of variants that have EAS_AF no less than 0.5:", np.sum(loc_group_select), "\n")

number of variants that have EAS_AF no less than 0.5: 2861 



### Fisher's Exact Test

* `ref_count_noneas`: number of reference allele count in non-EAS population
* `alt_count_noneas`: number of alternate allele count in non-EAS population
* `ref_count_eas`: number of reference allele count in EAS population
* `alt_count_eas`: number of alternate allele count in EAS population

In [0]:
if "gt" in dir():
    del gt
    
csv_eas = csv[csv['Superpopulation code'] == 'EAS']
csv_non = csv[csv['Superpopulation code'] != 'EAS']

csv_eas_index = np.array(csv_eas['Sample name'])
csv_non_index = np.array(csv_non['Sample name'])

In [0]:
vcf_subjects = np.array(vcfzarr['11/samples'])
print(len(vcf_subjects))
vcf_subjects

2504


array(['HG00096', 'HG00097', 'HG00099', ..., 'NA21142', 'NA21143',
       'NA21144'], dtype=object)

In [0]:
index_eas = np.array([x in csv_eas_index for x in vcf_subjects])
index_non = np.array([x in csv_non_index for x in vcf_subjects])

In [0]:
gt_eas = gt_select[:, index_eas]
gt_eas   # the genotype of EAS

<GenotypeArray shape=(2861, 504, 2) dtype=int8>
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
0/1 0/0 0/0 0/0 1/1 ... 0/0 0/0 1/1 1/0 1/1
1/1 1/0 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
...
1/1 1/0 1/1 0/0 0/1 ... 1/0 0/1 0/0 1/1 1/0
1/1 1/1 1/1 0/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
0/0 0/0 1/0 0/0 0/1 ... 1/0 1/1 0/0 0/1 1/0

In [0]:
count_eas = gt_eas.count_alleles()
count_eas

<AlleleCountsArray shape=(2861, 4) dtype=int32>
   8 1000    0    0
 501  507    0    0
 164  844    0    0
...
252 756   0   0
 12 996   0   0
460 548   0   0

In [0]:
gt_non = gt_select[:, index_non]
gt_non    # the genotype of NON-EAS

<GenotypeArray shape=(2861, 2000, 2) dtype=int8>
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 1/1 0/0
1/1 0/0 0/0 0/0 0/1 ... 1/1 1/1 1/0 1/1 0/0
...
1/1 0/0 0/1 0/0 1/1 ... 0/1 1/1 0/1 1/1 1/1
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 1/1 1/1
1/1 1/1 0/1 1/0 1/1 ... 1/1 1/1 1/1 1/1 1/1

In [0]:
count_non = gt_non.count_alleles()
count_non

<AlleleCountsArray shape=(2861, 4) dtype=int32>
 355 3645    0    0
3368  632    0    0
1891 2109    0    0
...
2632 1368    0    0
 106 3894    0    0
1492 2508    0    0

In [0]:
matrix4 = np.c_[np.array(count_non)[:, :2], np.array(count_eas)[:, :2]]
column_header = ['ref_count_noneas', 'alt_count_noneas', 'ref_count_eas', 'alt_count_eas']
table = pd.DataFrame(matrix4, columns = column_header)

variant_id = np.array(vcfzarr["11/variants/ID"])
table.index = np.array(variant_id)[loc_group_select]
table.head()    
# the first 5 rows of the table with required 4 columns

,ref_count_noneas,alt_count_noneas,ref_count_eas,alt_count_eas
rs371609562,355,3645,8,1000
rs574324672,3368,632,501,507
rs61869635,1891,2109,164,844
rs61869658,250,3750,0,1008
rs191800290,2864,1136,441,567


In [0]:
# add 1 to each count to get a table called table_corr to avoid error in chi2_contingency()
matrix4_corr = np.c_[np.array(count_non)[:, :2]+1, np.array(count_eas)[:, :2]+1]
column_header = ['ref_count_noneas', 'alt_count_noneas', 'ref_count_eas', 'alt_count_eas']
table_corr = pd.DataFrame(matrix4_corr, columns = column_header)

variant_id = np.array(vcfzarr["11/variants/ID"])
table_corr.index = np.array(variant_id)[loc_group_select]
# table_corr

Now, we can perform Chi-squared test on each row and obtain the significance ( _i.e._ chi-square and P-value) of the distribution. Refer to the [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) function.

In [0]:
# gt is not useful anymore, clear it from memory
if "gt" in dir():
    del gt

chi2test = table_corr.apply(lambda x: chi2_contingency(np.array(x).reshape((2, 2))), axis=1)

chi2result = pd.DataFrame(chi2test)
chi2result['p_value'] = chi2test.apply(lambda x: x[1])
chi2result['chi2'] = chi2test.apply(lambda x: x[0])

chi2result_sorted = chi2result.sort_values(by='p_value')
# del chi2result_sorted['0']
chi2result_sorted.head()

,0,p_value,chi2
rs7479605,"(1604.5085014617066, 0.0, 1, [[3110.0937749401...",0.000000e+00,1604.508501
rs10832878,"(1211.8151127350257, 1.650273623409321e-265, 1...",1.650274e-265,1211.815113
rs7124394,"(1179.4207732069854, 1.8103869172441616e-258, ...",1.810387e-258,1179.420773
rs77537847,"(1140.3993209513342, 5.4760255193711704e-250, ...",5.476026e-250,1140.399321
rs11246324,"(1112.7523499422887, 5.587901389877683e-244, 1...",5.587901e-244,1112.752350


According to the table sorted by P-value above, the top 5 variant ID that have significane distinct distribution in EAS are: 

rs7479605	
rs10832878	
rs7124394	
rs77537847	
rs11246324	

## Clinically relavant variants

In [0]:
# !gzip -cd /content/drive/My\ Drive/data/clinvar_20190909.vcf.gz | head -n 100

In [1]:
clinvar_file = DATA /"clinvar_20190909.vcf.gz"

def get_vcfvep_header(filename):
    with gzip.open(filename, "rb") as fi:
        for l in fi:
            l = l.decode("utf-8")
            if l.startswith("##"):
                continue
            elif l.startswith("#"):
                return l[1:].strip().split("\t")
            else:
                raise ValueError("Something is wrong in the file!")
                
clinvar_header = get_vcfvep_header(clinvar_file)

clinvar = pd.read_csv(clinvar_file, comment='#', sep="\t")
clinvar.columns = clinvar_header

clinvar.head()

In [0]:
clinvar.INFO.unique()

array(['AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;ALLELEID=446939;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.10:g.949422G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001583|missense_variant;ORIGIN=1;RS=143888043',
       'ALLELEID=626469;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.10:g.949427A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001583|missense_variant;ORIGIN=1',
       'ALLELEID=626470;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.10:g.949491G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=U

Filter out the clinvar variants to keep only the pathogenic ones and the variants in chromosome 11.

In [0]:
clinvar_c11 = clinvar[(clinvar.CHROM == 11)]
clinvar_c11.head()

# extract CLINSIG field from INFO column
clinvar_c11patho = clinvar_c11[clinvar_c11.INFO.str.contains('CLNSIG=Pathogenic')]
clinvar_c11patho.head()

# # another way to extract CLINSIG field from INFO column
# clinvar['clinsig'] = clinvar.INFO.apply(lambda i: dict([field.split('=') for field in i.split(';')])['CLNSIG'])
# clinvar.clinsig.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
242774,11,299372,183677,G,A,.,.,"ALLELEID=181576;CLNDISDB=MedGen:C2931093,OMIM:..."
242783,11,299504,37143,G,A,.,.,"ALLELEID=45805;CLNDISDB=MedGen:C2931093,OMIM:6..."
242832,11,533308,522672,C,CG,.,.,"ALLELEID=513316;CLNDISDB=MedGen:C0587248,OMIM:..."
242841,11,533467,12607,C,T,.,.,ALLELEID=27646;CLNDISDB=Human_Phenotype_Ontolo...
242863,11,533553,12605,T,C,.,.,"ALLELEID=27644;CLNDISDB=MedGen:C0587248,OMIM:2..."


In [0]:
clinvar_c11patho.shape

(4093, 8)

There are 4093 clinvar variants.